<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CART-算法" data-toc-modified-id="CART-算法-1">CART 算法</a></span></li><li><span><a href="#1.-CART-生成" data-toc-modified-id="1.-CART-生成-2">1. CART 生成</a></span><ul class="toc-item"><li><span><a href="#1.1-回归树生成" data-toc-modified-id="1.1-回归树生成-2.1">1.1 回归树生成</a></span><ul class="toc-item"><li><span><a href="#最小二乘回归树生成算法" data-toc-modified-id="最小二乘回归树生成算法-2.1.1">最小二乘回归树生成算法</a></span></li></ul></li><li><span><a href="#1.2-分类树生成" data-toc-modified-id="1.2-分类树生成-2.2">1.2 分类树生成</a></span><ul class="toc-item"><li><span><a href="#基尼指数" data-toc-modified-id="基尼指数-2.2.1">基尼指数</a></span></li><li><span><a href="#CART-生成算法" data-toc-modified-id="CART-生成算法-2.2.2">CART 生成算法</a></span></li></ul></li></ul></li><li><span><a href="#参考文献" data-toc-modified-id="参考文献-3">参考文献</a></span></li></ul></div>

相关文章：

[机器学习 | 目录](https://blog.csdn.net/weixin_45488228/article/details/99691709)

[监督学习 | ID3 决策树原理及Python实现](https://blog.csdn.net/weixin_45488228/article/details/98665115)

[监督学习 | ID3 & C4.5 决策树原理](https://blog.csdn.net/weixin_45488228/article/details/102136999)

[监督学习 | 决策树之Sklearn实现](https://blog.csdn.net/weixin_45488228/article/details/98751821)

[监督学习 | 决策树之网络搜索](https://blog.csdn.net/weixin_45488228/article/details/99253498)

本文大部分内容搬运自李航老师的《统计学习方法》<sup>[1]</sup>，以给出决策树算法较为完整的定义，关于决策树算法的 Sklearn 实现，可以参考这篇[文章](https://blog.csdn.net/weixin_45488228/article/details/98751821)。

# CART 算法

`分类与回归树`（classification and regression tree, CART）模型由 Beriman 等人在 1984 年提出，是应用广泛的决策树学习方法，CART 同样由特征选择、树的生成及剪枝组成，既可以用于分类也可以用于回归，以下将用于分类与回归的树统称为决策树。

CART 是在给定输入随机变量 X 条件下输出随机变量 Y 的条件概率分布的学习方法。CART 假设决策树是二叉树，内部结点特征的取值为“是”和“否”，左分支是取值为“是”的分支，右分支是取值为“否”的分支。这样的决策树等价于递归地二分每个特征，将输入空间即特征空间划分为有限个单元，并在这些单元上确定预测的概率分布，也就是在输入给定的条件下输出的条件概率分布。

CART 算法由以下两部组成：

（1）决策树生成：基于训练数据集生成决策树，生成的决策树要尽量大；

（2）决策树剪枝：用验证数据集对已生成的树进行剪枝并选择最优子树，这是用损失函数最小作为剪枝的标准。

# 1. CART 生成

决策树的生成就是递归地构建二叉决策树的过程。对`回归树`用`平方误差`最小化准则，对`分类树`用`基尼指数`（Gini index）最小化准则，进行特征选择，生成二叉树。

## 1.1 回归树生成

假设 $X$ 与 $Y$分别是输入和输出变量，并且 $Y$ 是连续变量，给定训练数据集：

$$D=\{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\}$$

一个回归树对应着输入空间（即特征空间）的一个划分以及在划分的单元上的输出值。假设已将输入空间划分为 $M$ 个单元 $R_1,R_2,...,R_M$ 并且在每个单元 $R_m$ 上有一个固定的输出值 $C_m$，于是回归树模型可表示为：

$$f(x)=\sum_{m=1}^M c_mI(x \in R_m) \tag{1}$$

当输入空间的划分确定时，可以用平方误差 $\sum_{x_i\in R_m}(y_i-f(x_i))^2$ 来表示回归树对于训练数据的预测误差，用平方误差最小的准则求解每个单元上的最优输出值。

因此，单元 $R_m$ 上的 $c_m$ 的最优值 $\hat{c}_m$ 是 $R_m$ 上所有输入实例 $x_i$ 对应的输出 $y_i$ 的均值，即：

$$\hat{c}_m = ave(y_i|x_i \in R_m) \tag{2}$$

这里采用启发式的方法对输入空间进行划分：选择第 $j$ 个变量 $x^{(j)}$ 和它取的值 $s$，作为切分变量（splitting variable）和切分点（splitting point），并定义两个区域：

$$R_1(j,s)=\{x|x^{(j)}\leq s\} \quad 和 \quad R_2(j,s)=\{x|x^{(j)}> s\} \tag{3}$$

然后寻找最优切分变量 $j$ 和最优切分点 $s$：

$$\min \limits_{j}\bigg[ \min \limits_{c_j} \sum_{x_i\in R_1(j,s)}(y_i-c_1)^2 + \min \limits_{c_j} \sum_{x_i\in R_2(j,s)}(y_i-c_2)^2
\bigg] \tag{4}$$

对固定输入变量 $j$ 可以找到最优切分点 $s$。

因此有：

$$\hat{c}_1=ave(y_i|x_i\in R_1(j,s)) \quad 和 \quad \hat{c}_2=ave(y_i|x_i\in R_2(j,s)) \tag{5}$$

遍历所有输入变量，找到最优的切分变量 $j$，构造一个对 $(j,s)$。依次将输入空间划分为两个区域。接着，最每个区域重复上述划分过程，直到满足停止条件为止，这样就生成一颗回归树。这样的回归树通常称为`最小二乘回归树`（least squares regression tree）。

### 最小二乘回归树生成算法

**输入**：训练数据集 $D$;

**输出**：回归树 $f(x)$.

在训练数据集所在的输入空间中，递归地将每个区域划分为两个子区域并决定每个子区域熵的输出值，构建二叉决策树；

（1）选择最优切分变量 $j$ 和最优切分点 $s$，求解：

$$\min \limits_{j}\bigg[ \min \limits_{c_j} \sum_{x_i\in R_1(j,s)}(y_i-c_1)^2 + \min \limits_{c_j} \sum_{x_i\in R_2(j,s)}(y_i-c_2)^2
\bigg] \tag{6}$$

$\quad$ 遍历变量 $j$，对固定的切分变量 $j$ 扫描切分点 $s$，选择使上式达到最小的对 $(j,s)$；

（2）用选定的对 $(j,s)$ 划分区域并决定相应的输出值：

$$R_1(j,s)=\{x|x^{(j)}\leq s\} \quad R_2(j,s)=\{x|x^{(j)}> s\} \tag{7}$$

$$\hat{c}_m=\frac{1}{N_m} \sum_{x_i \in R_m(j,s)}y_i,\quad x\in R_m,\quad m=1,2 \tag{8}$$

（3）继续对两个子区域调用步骤 (1)，(2)，直至满足停止条件；

（4）将输入空间划分为 $M$ 个单元 $R_1,R_2,...,R_M$ ，生成决策树：

$$f(x)=\sum_{m=1}^M c_mI(x \in R_m) \tag{9}$$

## 1.2 分类树生成

分类树用基尼指数选择最优特征，同时决定该特征的最优二值切分点。

### 基尼指数

分类问题中，假设有 $K$ 个类，样本点属于第 $k$ 类的概率为 $p_k$，则概率分布的基尼指数定义为：

$$Gini(p)=\sum_{k=1}^Kp_k(1-p_k)=1-\sum_{k=1}^Kp_k^2 \tag{10}$$

对于二类分类问题，若样本点属于第 1 个类的概率是 $p$，则概率分布的基尼指数为：

$$Gini(p)=2p(1-p) \tag{11}$$

对于给定的样本集合 $D$，其基尼指数为：

$$Gini(D)=1-\sum_{k=1}^K\bigg(\frac{|C_k|}{|D|} \bigg)^2 \tag{12}$$

这里，$C_k$ 是 $D$ 中属于第 $k$ 类的样本子集，$K$ 是类的个数。

如果样本集合 $D$ 根据特征 $A$ 是否取某一可能值 $\alpha$ 被分割成 $D_1$ 和 $D_2$ 来那个部分，即：

$$D_1=\{(x,y)\in D|A(x)=a\}, \quad D_2=D-D_1 \tag{13}$$

则在特征 $A$ 的条件下，集合 $D$ 的基尼指数定义为：

$$Gini(D,A)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2) \tag{14}$$

基尼指数 $Gini(D)$ 表示集合 $D$ 的不确定性，**基尼指数值越大，样本集合的不确定性也就越大**，这一点与熵相似。

### CART 生成算法

**输入**：训练数据集 $D$，停止计算的条件；

**输出**：CART 分类决策树。

根据训练数据集，从根结点开始，递归地对每个结点进行以下操作，构建二叉决策树：

（1）设结点的训练数据集为 $D$，计算现有特征对该数据集的基尼指数。此时，对每一个特征 $A$，对其可能取的每个值 $a$，根据样本点对 $A=a$ 的测试为“是”或“否”将 $D$ 分割成 $D_1$ 和 $D_2$两部分，利用式 (14) 计算 $A=a$ 时的基尼指数；

（2）在所有可能的特征 $A$ 以及它们所有可能的切分点 $a$ 中，选择基尼指数最小的特征及其对应的切分点作为最优特征与最优切分点。依最优特征与最优切分点，从现生成两个子结点，将训练数据集依特征分配到两个子结点中去；

（3）对两个子结点递归地调用 (1) ，(2)，直到满足停止条件；

（4）生成 CART 决策树。

算法停止计算的条件是结点中的样本个数小于预定阈值，或样本集的基尼指数小于预定阈值（样本基本属于同一类），或者没有更多特征。

# 参考文献

[1] 李航. 统计学习方法[M]. 北京: 清华大学出版社, 2012: 55-66.